In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

     |████████████████████████████████| 904 kB 5.1 MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelea

In [ ]:
import requests
from lxml import html, etree
from selenium.webdriver import Chrome, ChromeOptions
import random
import time
from datetime import datetime, timedelta
import csv
from pandas import DataFrame
import pandas as pd
import random

In [ ]:
def data_writer(li):
    with open(r'output.csv', mode="a",
              newline="", encoding="utf-8-sig") as f_out:
        my_writer = csv.writer(f_out, delimiter=',')
        my_writer.writerow(li)
data_writer(['URL', "Title", "Location",
            "No. of Reviews", "Rating", "Price Range", "Cuisine"])

In [ ]:
BASE = 'https://www.tripadvisor.com'

##Chrome Driver Options##
width = range(500, 700, 10)
height = range(800, 1000, 20)
chrome_options = ChromeOptions()
chrome_options.add_argument(
    f"window-size={random.choice(width)},{random.choice(height)}")
chrome_options.add_argument('--no-default-browser-check')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-default-apps')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--log-level=3')
chrome_options.add_argument('--headless')
wd = Chrome('chromedriver',options=chrome_options)
wd.get('https://www.tripadvisor.com/Restaurants-g187849-Milan_Lombardy.html')
try:
    wd.find_element_by_xpath('//button[@id="_evidon-accept-button"]').click()
except:
    pass
time.sleep(2)
wd.find_element_by_xpath('//span[contains(text(), "Takeout")]').click()
time.sleep(3)
tree = html.fromstring(wd.page_source)

In [ ]:
for item in tree.xpath('//div[@data-component-props="page-manifest"]/div//span/text()'):
    try:
        num_results = int(item)
        break
    except:
        pass
a_li = [0]
a = 0
for i in range(int(num_results/30)):
    a += 30
    a_li.append(a)

In [ ]:
headers = {
    'authority': 'www.tripadvisor.com',
    'sec-ch-ua': '"Chromium";v="92", " Not A;Brand";v="99", "Google Chrome";v="92"',
    'x-requested-with': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36',
    'accept': '*/*',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.tripadvisor.com/Restaurants-g187849-Milan_Lombardy.html',
    'accept-language': 'en-US,en;q=0.9,ar;q=0.8',
    "cookie":''
}

page_counter = 1
restaurant_urls_li = []
for a in a_li:
    params = (
    ('Action', 'PAGE'),
    ('ajax', '1'),
    ('availSearchEnabled', 'true'),
    ('sortOrder', 'relevance'),
    ('geo', '187849'),
    ('itags', '10591'),
    ('zfp', '10601'),
    ('eaterydate', datetime.today().strftime('%Y_%m_%d')),
    ('date', (datetime.today()+timedelta(days=1)).strftime('%Y-%m-%d')),
    ('time', '20:00:00'),
    ('people', '2'),
    ('o', f'a{a}'),
    )
    r = requests.get(f'https://www.tripadvisor.com/RestaurantSearch', params=params, headers=headers)
    tree = html.fromstring(r.content)
    list_of_restaurants = tree.xpath('//div[contains(@data-test,"list_item")]')
    for restaurant in list_of_restaurants:
        restaurant_url = f"{BASE}{restaurant.xpath('.//a')[0].attrib['href']}"
        restaurant_urls_li.append(restaurant_url)
    print(f'Page No. {page_counter} scrapped')
    page_counter += 1

Page No. 1 scrapped
Page No. 2 scrapped
Page No. 3 scrapped
Page No. 4 scrapped
Page No. 5 scrapped
Page No. 6 scrapped
Page No. 7 scrapped
Page No. 8 scrapped
Page No. 9 scrapped
Page No. 10 scrapped
Page No. 11 scrapped
Page No. 12 scrapped
Page No. 13 scrapped
Page No. 14 scrapped
Page No. 15 scrapped
Page No. 16 scrapped
Page No. 17 scrapped
Page No. 18 scrapped
Page No. 19 scrapped
Page No. 20 scrapped
Page No. 21 scrapped
Page No. 22 scrapped
Page No. 23 scrapped
Page No. 24 scrapped
Page No. 25 scrapped
Page No. 26 scrapped
Page No. 27 scrapped
Page No. 28 scrapped
Page No. 29 scrapped
Page No. 30 scrapped
Page No. 31 scrapped
Page No. 32 scrapped
Page No. 33 scrapped
Page No. 34 scrapped
Page No. 35 scrapped
Page No. 36 scrapped
Page No. 37 scrapped
Page No. 38 scrapped
Page No. 39 scrapped
Page No. 40 scrapped
Page No. 41 scrapped
Page No. 42 scrapped
Page No. 43 scrapped
Page No. 44 scrapped
Page No. 45 scrapped
Page No. 46 scrapped
Page No. 47 scrapped
Page No. 48 scrapped
P

In [ ]:
print(f'You have {len(restaurant_urls_li)} restaurants.')

You have 3021 restaurants.


In [ ]:
list_saved = DataFrame(restaurant_urls_li)
list_saved.to_csv(r"list_saved.csv")
restaurant_urls_li = pd.read_csv(r"list_saved.csv")
restaurant_urls_li = restaurant_urls_li.iloc[:,-1:]
restaurant_urls_li = restaurant_urls_li.values.tolist()

In [ ]:
num_rest = len(restaurant_urls_li)
rest_x_piece = 300
value = random.randint(0,10)
print (value)
if value == 0:
  start = rest_x_piece * 0
  end = rest_x_piece * 1
elif value == 1:
  start = rest_x_piece * 1
  end = rest_x_piece * 2
elif value == 2:
  start = rest_x_piece * 2
  end = rest_x_piece * 3
elif value == 3:
  start = rest_x_piece * 3
  end = rest_x_piece * 4
elif value == 4:
  start = rest_x_piece * 4
  end = rest_x_piece * 5
elif value == 5:
  start = rest_x_piece * 5
  end = rest_x_piece * 6
elif value == 6:
  start = rest_x_piece * 6
  end = rest_x_piece * 7
elif value == 7:
  start = rest_x_piece * 7
  end = rest_x_piece * 8
elif value == 8:
  start = rest_x_piece * 8
  end = rest_x_piece * 9
elif value == 9:
  start = rest_x_piece * 9
  end = rest_x_piece * 10
elif value == 10:
  start = rest_x_piece * 10
  end = num_rest
start = int(start)
end = int(end)
print (start)  
print (end)         

6
1800
2100


In [ ]:
for restaurant_url in restaurant_urls_li[start:end]:
    wd.get(restaurant_url[0])
    tree = html.fromstring(wd.page_source)
    restaurant = tree.xpath('//div[contains(@id,"taplc_top_info")]')[0]
    try:
        restaurant_name = restaurant.xpath('.//h1')[0].text
        for item in restaurant.xpath('.//a'):
            try:
                item_href = item.attrib['href']
                if item_href == '#MAPVIEW':
                    restaurant_location = item.text
                if item_href == '#REVIEWS':
                    restaurant_reviews_num = item.xpath('.//span')[0].text
                    restaurant_reviews_rating = item.xpath(
                        './/svg')[0].attrib['title'].split(' ')[0]
            except:
                pass
        try:
            price = tree.xpath(
                '//div[contains(text(),"PRICE RANGE")]/following-sibling::div')[0].text
        except:
            price = ''
        try:
            cuisine = tree.xpath(
            '//div[contains(text(),"CUISINES")]/following-sibling::div')[0].text
        except:
            cuisine = ''
        data_writer([restaurant_url, restaurant_name, restaurant_location,
                    restaurant_reviews_num, restaurant_reviews_rating, price, cuisine])
        print(f'{restaurant_name} Scrapped.')
    except:
        print(restaurant_url)